## 추진배경

* 부서원 모두의 공통 업무이며, 여러가지 불편사항이 존재하나 자동화방안에 대한 기존 고민X
  * 필요에 의해 ID가 3개로 나뉘어있으며, 공동인증서 활용으로 타이핑작업 및 ID/PW/공동인증서PW의 관리 및 입력 불편함
  * 여러 건의 COO를 발급하고 대응하지만, 각 업무는 COO 1건별로 메뉴에 진입하여 수행해야함
    * 심사완료여부, 발급거절시 사유확인, 출력 및 사본 저장 등
* 발급실적 담당자는 월마다 부서 전체의 발급실적을 관리하기위해 별도의 작업을 수행
  * 3개의 ID에 접속하여 20여개 페이지의 표를 복사하고, 중복/심사거절건 제거 등 수작업 가공 진행
* 위 2가지 문제를 해결할 방법에 대한 고민 및 해결방안 도출 : 데이터의 통합db화 및 편의기능 추가
  * 공용PC를 운영중인 부서로 주기적으로 활동하는 크롤러를 운영하여 db로 저장가능
  * streamlit을 활용해 db에 대한 검색과 확인 가능한 대시보드형 사이트 제작하여 일반유저도 손쉽게 사용
  * 희망시 심사번호를 streamlit사이트에 등록하여 대응상황 발생시 toast알림이 가도록 제작
  * 축적된 db에서 월 데이터를 추출할 수 있도록 버튼을 누르면 SQL쿼리 후 Excel저장기능 추가
  * 3개의 ID에 대한 접속버튼으로 자동로그인 기능 추가
    * streamlit사이트의 정보관리메뉴에서 접속ID/PW등을 수정하면 공용PC의 json파일을 수정하여 모든유저에게 반영

>
**[용어설명]**<br>
**COO** : 원산지증명서, Country of Origin<br>

## 효과
* 크롤러, db구축, 관리 및 편의기능을 포함한 대시보드형 사이트 구축으로 기존의 불편사항들을 해결
  * 자동로그인, 대응사항 toast알림으로 지속적인 새로고침 등 불필요한 작업제거
  * 월마다 진행되던 불필요한 데이터 가공작업 제거(SQL쿼리 월 조건등은 사용자가 strealit사이트에서 수정 가능)
* 기존 발급거절 사유의 db화로 주요 케이스에 대한 분석 및 사전대응계획 수립가능
  * 주요 케이스는 사전에 신고한 제품의 단위와 신청시스템 단위의 차이로, 관련 담당자에 개선제언 예정

## github repository
[관련 github레포](https://github.com/KR9268/-P-MonitoringCOO)

## [세부내용] 구현내용 & 사용한 언어/패키지 등
(1) 저장할 db는 `sqlite3`으로 테이블 생성, 컬럼지정 등을 수행함 (컬럼별 조건은 하단 참조)
  * 접수번호 varchar PRIMARY KEY ,<br> → **대표Invoice와 고민했는데, 100% 유일값이라 Primary로 지정** 
     증명서종류 varchar, <br>
     대표Invoice varchar(10),<br> 
     접수일시 datetime, <br>
     처리상태 varchar, <br>
     Remark varchar
(2) 각 기능은 아래의 파일로 나누어 개인/공용PC에서 실행
  * **MonitoringCOO(기본파일)** : `streamlit`활용한 UI, json/pickle파일 읽기, 유저의 자동로그인, 월추출 데이터 저장 등
    * 마지막 스크레핑 시점을 표기하여 얼마나 최신화된 데이터인지 유저에게 공유
  * **MonitoringCOO_crawler** : `selenium`으로 스크레핑, 스크레핑작업에 필요한 로그인 기능(`pyautogui`, `pywin32`로 이미지/키/윈도우 인식)
    * 유저가 **기본파일**에서 로그인기능을 사용하는 경우, 이 파일에서 import해서 사용하고 **코드는 여기서 통합관리**
    * 스크레핑작업은 기본적으로 **Scheduler**파일에서 실행되지만, 필요시 이 파일을 실행하여 수동 스크레핑 (**코드는 여기서 통합관리**)
  * **MonitoringCOO_push** : **기본파일**에서 유저가 등록해둔 대표Invoice번호를 db에서 조회하여, `win11toast`로 알림
                         (처음에는 파이썬과 호환성/속도가 좋은 pickle/list로 관리하고자 했으나, 사용자ID등 추가정보 관리가 필요하여 json/dict로 관리)
  * **Scheduler** : 스크레핑 주기/시간을 관리하는 파일. 주로 공용PC에서 작업
                    (9~17시 이후엔 데이터변경이 없으므로 이 시간대에만 작동하도록 설정, 서버설정 등을 고려하여 작업주기 반영 예정)